# R web services with Azure Machine Learning Services (not Studio ;-))
This notebook shows how to write and deploy a scoring webservice with Azure Machine Learning Services (AMLS) running R code.

The underlying scenario is a **scoring** scenario. If your scenario is rather compute-intensive **batch computation**, you might rather wanna use the notebook [here](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/training/train-on-amlcompute/train-on-amlcompute.ipynb) as base and adopt some of the ideas contained in this notebook (eg. using rpy2). Similarly, for creating **pipelines**, you can use the samples [here](https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/machine-learning-pipelines) as base.

# Get workspace
Load existing workspace from the config file info. If you have not prepared the config file yet, use the `configuration.ipynb` notebook from the official [samples repo](https://github.com/Azure/MachineLearningNotebooks) to create it. Alternatively you rewrite the code to instantiate the Workspace instance directly, eg.:
*workspace = Workspace(subscription_id=subscription_id, resource_group=resource_group,workspace_name=workspace_name)*

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# Register the model
Register an existing trained model, add description and tags.

Note: see the included `create_model.r` file so see how a model could be created.

In [ ]:
from azureml.core.model import Model
model = Model.register(model_path = "model.RData",
                       model_name = "model.RData",
                       tags = {'area': "samples", 'type': "regression"},
                       description = "A simple linear regression model to show the usage of R in Azure Machine Learning Services.",
                       workspace = ws)

print(model.name, model.description, model.version)

# Create an image
Create an image using
- the registered model (created and registered above),
- a script that will load and run the model and
- all dependencies required by the codes.

In [ ]:
%%writefile score.py
import json
from azureml.core.model import Model
import rpy2.rinterface
import rpy2.robjects as robjects

def init():
    # note: this function is run whenever the container is started
    # init rpy2
    rpy2.rinterface.initr()
    # load model
    model_path = Model.get_model_path('model.RData')
    robjects.r("load('{model_path}')".format(model_path=model_path))
    # run init() function in R (if exists)
    robjects.r("if (exists('init', mode='function')) { init() }")

def run(input_json_string):
    # note: this function is run whenever a scoring request has been received
    try:
        result_vector = robjects.r(
                "run('{input_json_string}')".format(input_json_string=input_json_string)
            )
        if len(result_vector) > 0:
            try:
                return json.loads(result_vector[0])
            except ValueError:
                return {"message": result_vector }
    except Exception as e:
        error = str(e)
        return {"error": error }

Define the dependencies required by the scoring script to run.

- rpy2 is a package to run R from Python
- mro-base is Microsoft's R distribution
- r-jsonlite and r-essentials are two conda packages which add the jsonlite package and a few other common packages to our R

If you need more packages, you can specify them with the conda_packages parameter below - which should be the preferred method.

However, if your packages are not available on conda, you can also tell the image_configuration() function below to add some local files or folders to the image (parameter: dependencies) and then install the packages manually by specifying some Docker commands in a file passed to the docker_file parameter. (The files/folders specified by dependencies parameter will be put into /var/azureml-app. The docker_file is run before the conda_packages. Hence you might to install some conda packages on your own if they are required by some commands in docker_file.)

Another option would be to install additional packages within the R or Python code (in case the container will be connected to the required network). However in that case you may end up in unnecessary latencies. So installing all dependencies once in the image should always be preferred.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

conda_dependencies = CondaDependencies.create(conda_packages=['rpy2', 'mro-base','r-jsonlite','r-essentials'])

with open("conda_dependencies.yml","w") as f:
    f.write(conda_dependencies.serialize_to_string())

Create the image (this might take a while).

In [ ]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "conda_dependencies.yml",
                                                  description = "Sample image to show the usage of R with Azure Machine Learning Services.",
                                                  tags = {'area': "samples", 'type': "regression"},
                                                 )

image = ContainerImage.create(name = "r-on-amls-sample",
                              models = [model],
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

# Deployment

There is multiple engines where the image we created before could be deployed to. Select the one which is most appropriate for you.

## Option 1: Deploy to ACI

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice

myaci_config = AciWebservice.deploy_configuration(
    cpu_cores = 2, 
    memory_gb = 2,
    tags = {'area': "samples", 'type': "regression"},
    description = 'Sample image to show the usage of R with Azure Machine Learning Services.'
)

service_name = "r-on-aml-service-aci"
runtime = "python" 
driver_file = "score.py"
my_conda_file = "conda_dependencies.yml"

aci_service = Webservice.deploy_from_model(
  workspace=ws, 
  name=service_name,
  deployment_config = myaci_config,
  models = [model],
  image_config = myimage_config,
  enable_ssl = True
)
aci_service.wait_for_deployment(show_output=True)

print("Scoring URI: {}".format(aci_service.scoring_uri))
print("Swagger URI: {}".format(aci_service.swagger_uri))

## Option 2: Deploy to AKS (Kubernetes)


In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice

aks_cluster_name = "aks-cluster"
aks_cluster_agent_count = 3
aks_cluster_vm_size = "Standard_D3_v2"

try:
    aks_cluster = ComputeTarget(workspace=workspace, name=aks_cluster_name)
    print("Cluster '{}' exists already.".format(aks_cluster_name))
except ComputeTargetException:
    print("Cluster '{}' does not exist yet.".format(aks_cluster_name))
    print("Creating cluster...".format(aks_cluster_name))
    provisioning_configuration = AksCompute.provisioning_configuration(
        agent_count=aks_cluster_agent_count,
        vm_size=aks_cluster_vm_size
    )
    provisioning_configuration.enable_ssl(leaf_domain_label=aks_cluster_name)
    aks_cluster = ComputeTarget.create(
        workspace=ws,
        name=aks_cluster_name,
        provisioning_configuration=provisioning_configuration
    )
    aks_cluster.wait_for_completion(show_output=True)    
    print(aks_cluster.provisioning_state)
    print(aks_cluster.provisioning_errors)

In [ ]:
%%time

aks_target = ComputeTarget(workspace = ws, name = aks_cluster_name)
aks_config = AksWebservice.deploy_configuration()

aks_service_name ='r-on-aml-service'
aks_service = Webservice.deploy_from_image(workspace = ws, 
                                           name = aks_service_name,
                                           image = image,
                                           deployment_config = aks_config,
                                           deployment_target = aks_target)
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

In [ ]:
print("Scoring URI: {}".format(aks_service.scoring_uri))
print("Swagger URI: {}".format(aks_service.swagger_uri))
print("Keys: {}".format(aks_service.get_keys()))

## Option 3: Deploy to Azure Web Apps for Containers

This option is currently in preview. See [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-app-service) for more details.

# Test the web service
We test the web sevice by passing data.

In [ ]:
%%time
import json

test_sample = json.dumps({'x': 987.654})
test_sample = bytes(test_sample,encoding = 'utf8')

prediction = aks_service.run(input_data = test_sample)

print(prediction)

# Clean up
Delete the service, image and model.

In [ ]:
%%time
aks_service.delete()
image.delete()
model.delete()